In [2]:
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import sklearn


sample_submission.csv - A submission file in the correct format.
PassengerId - Id for each passenger in the test set.
Transported - The target. For each passenger, predict either True or False.


In [5]:
sample_submission_df = pl.read_csv('sample_submission.csv')
train_df = pl.read_parquet('train_df.parquet')
test_df = pl.read_parquet('test_df.parquet')

In [4]:
classifier = sklearn.ensemble.HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_iter=1000,
    max_leaf_nodes=None,
    max_depth=None,
    min_samples_leaf=10,
    l2_regularization=1.0,
    categorical_features="from_dtype",
    warm_start=True,
    scoring="f1",
    validation_fraction=0.2,
    verbose=1,
    random_state=42,
)